## 식수 예측
- 유형 : 회귀
- 평가지표 : MAE
- LightGBM 기본 모델
- 활용 변수 : 연도월, 요일, 미출근(휴가자수, 출장자수, 재택근무자수의 합), 추가근무건수, 중식_메인, 석식_메인

In [32]:
# 라이브러리
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import koreanize_matplotlib
%config InlineBackend.figure_format = 'retina'

from glob import glob

# 시각화 테마 설정
sns.set_style('dark')
plt.rc('font', family='NanumGothic')
plt.rc('axes', unicode_minus=False)

# 파일 불러오기
files = glob('data/*.csv')
train = pd.read_csv(files[2])
test = pd.read_csv(files[0])
submit = pd.read_csv(files[3])
holiday = pd.read_csv(files[1])

# 복사 
n_train = train.copy()
n_test = test.copy()

# 미출근자 : 출장, 재택, 휴가자 수의 합
notinoffice = n_train.iloc[:,3:6].sum(axis=1)

# train, test에 미출근 파생변수 생성
n_train['미출근'] = notinoffice
n_test['미출근'] = notinoffice

# 출근자 파생변수 생성
n_train['출근'] = n_train['본사정원수'] - n_train['미출근']
n_test['출근'] = n_test['본사정원수'] - n_test['미출근']

# 석식계가 0인 날 구하고 인덱스 추출
no_dinner = n_train[n_train['석식계'] == 0]
no_dinner_index = no_dinner.index

# 석식메뉴가 없는 날 -> 'None' 채워넣기
# 어쨌든 추후에 범주형으로 인코딩 진행할 시에 동일한 값들은 동일한 범주로 묶일 것이기 때문에
# 비어있는 것의 영향력과 비어있는 것에 다른 값들과 겹치지 않는 값을 채워 이를 하나로 묶어준 
# 범주의 영향력(혹은 가중치)이 다르지 않을 것이라고 판단하고 진행
n_train.loc[n_train.index.isin(no_dinner_index), '석식메뉴'] = 'None'

# 활용할 컬럼
n_columns = ['일자', '요일', '출근', '미출근', '본사시간외근무명령서승인건수', '중식메뉴', '석식메뉴', '중식계', '석식계']

# 활용할 컬럼 train, test 적용
n_train = n_train[n_columns]
n_test = n_test[n_columns[:-2]]

# '본사시간외근무명령서승인건수' -> '추가근무건수'
n_train = n_train.rename(columns={'본사시간외근무명령서승인건수':'추가근무건수'})
n_test = n_test.rename(columns={'본사시간외근무명령서승인건수':'추가근무건수'})

# 요일 변수 인코딩
n_train['요일'] = n_train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
n_test['요일'] = n_test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

# 일자 데이터에서 여러 파생변수 생성
n_train['일자'] = pd.to_datetime(n_train['일자'])
n_test['일자'] = pd.to_datetime(n_test['일자'])

# 연도
n_train['연도'] = n_train['일자'].dt.year
n_test['연도'] = n_test['일자'].dt.year

# 월
n_train['월'] = n_train['일자'].dt.month
n_test['월'] = n_test['일자'].dt.month

# 연도월
n_train['연도월'] = n_train['일자'].astype(str).str[:7]
n_test['연도월'] = n_test['일자'].astype(str).str[:7]

# 2차 컬럼 선정
n_train = n_train[['일자', '연도', '월', '연도월', '요일', '출근', '미출근', '추가근무건수', '중식메뉴', '석식메뉴', '중식계', '석식계']]

# 메뉴는 train과 test를 합쳐서 일괄 전처리
menu_col = ['중식메뉴', '석식메뉴']

menu_train = n_train[menu_col]
menu_test = n_test[menu_col]

menu = pd.concat([menu_train, menu_test], ignore_index=True)

# 닫는 괄호가 없는 컬럼 전처리
menu['중식메뉴'][1003] = '카레라이스 (쌀:국내산,돈육:국내) 미소시루  감자만두*양념  애기새송이버섯볶음  골뱅이야채무침  포기김치 (김치:국내산)'

# 정규표현식
import re

def preprocessing(text):
    # 한글, 영문, 숫자만 남기고 모두 제거
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 원산지 표기 제거
    # text = re.sub('[\S]+산', ' ', text)
    # NEW 제거
    # text = re.sub('N|E|W', ' ', text)
    # new 제거
    # text = re.sub('n|e|w', ' ', text)
    # 120명 제거
    # text = re.sub('120명', ' ', text)
    # 1컷팅해서 제거
    # text = re.sub('1컷팅해서', ' ', text)
    # 괄호 포함된 문자 제거
    # text = re.sub('\s*\([^)]*\)', ' ', text)
    # 괄호 포함된 영어 외의 문자만 제거 : 신메뉴 표기 '(New)'는 남겨두기 위함
    text = re.sub('\((?![a-zA-Z]+\)).*?\)', ' ', text)
    text = re.sub('\dㄱ-ㅎㅏ-ㅣ가-힣\s]+$', ' ', text)
    # 중복으로 생성된 공백값 제거
    text = re.sub('[\s]+', ' ', text)
    return text

from tqdm import tqdm
tqdm.pandas() 

# map을 통해 전처리 일괄 적용
menu['중식메뉴'] = menu['중식메뉴'].progress_map(preprocessing)
menu['석식메뉴'] = menu['석식메뉴'].progress_map(preprocessing)

# 양쪽 끝에 ' '공백으로 인한 에러 방지
menu['중식메뉴'] = menu['중식메뉴'].str.strip()
menu['석식메뉴'] = menu['석식메뉴'].str.strip()

menu['중식메뉴'] = menu['중식메뉴'].str.split(' ')
menu['석식메뉴'] = menu['석식메뉴'].str.split(' ')

# 메뉴를 카테고리화하는 함수
def menu_split(df, col):
    '''
    df(데이터프레임)과 col(컬럼이름)을 넣으면
    인덱스를 기준으로 밥, 국, 메인으로 나눈 후
    파생변수를 생성해주는 함수

    * 메뉴가 없는 날을 고려하여, len(row)가
    밥, 국, 메인을 합한 길이인 3미만인 행은 값을
    원본 그대로 적용
    '''
    rice = [row[0] if len(row) > 2 else row[0] for row in df[col].values]
    soup = [row[1] if len(row) > 2 else row[0] for row in df[col].values]
    main = [row[2] if len(row) > 2 else row[0] for row in df[col].values]
    df[f'{col[:2]}_밥'] = rice
    df[f'{col[:2]}_국'] = soup
    df[f'{col[:2]}_메인'] = main
    return df

# 중식, 석식 적용
menu_split(menu, '중식메뉴')
menu_split(menu, '석식메뉴')

# train에 메뉴 카테고리화한 프레임 병합
train_menu = menu.iloc[:1205, 2:]
n_train = pd.concat([n_train, train_menu], axis=1)
n_train = n_train.drop(columns=['중식메뉴', '석식메뉴'])

# 학습데이터에서 석식계 0인 행 제외
n_train = n_train.drop(no_dinner_index, axis=0)
n_train = n_train.reset_index(drop=True)
n_train = n_train.drop(columns=['일자', '연도', '월', '출근', '석식_밥', '석식_국', '중식_밥', '중식_국'])

# test에 메뉴 카테고리화한 프레임 병합
test_menu = menu.iloc[1205:, 2:]
test_menu = test_menu.reset_index(drop=True)
n_test = pd.concat([n_test, test_menu], axis=1)
n_test = n_test.drop(columns=['중식메뉴', '석식메뉴'])
n_test = n_test.drop(columns=['일자', '연도', '월', '출근', '석식_밥', '석식_국', '중식_밥', '중식_국'])

# 라벨인코딩으로 개별 메뉴 카테고리화
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
n_train['중식_메인'] = le.fit_transform(n_train['중식_메인'])
n_train['석식_메인'] = le.fit_transform(n_train['석식_메인'])

n_test['중식_메인'] = le.fit_transform(n_test['중식_메인'])
n_test['석식_메인'] = le.fit_transform(n_test['석식_메인'])

# 연도월 카테고리화
n_train['연도월'] = n_train['연도월'].astype('category').cat.codes
n_test['연도월'] = n_test['연도월'].astype('category').cat.codes

# 변수 순서 맞춰주기
n_test = n_test[['연도월', '요일', '미출근', '추가근무건수', '중식_메인', '석식_메인']]

# 타겟값 지정
label = ['중식계', '석식계']

# 모델 생성
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X = n_train.drop(columns=label)
y1 = n_train[label[0]]
y2 = n_train[label[1]]

# 중식계 학습
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size=.2, random_state=42)
model1 = LGBMRegressor(random_state=42)
model1.fit(X_train, y_train)
print("Model1 Score: ",model1.score(X_test,y_test))
print("MAE:",mean_absolute_error(y_test, model1.predict(X_test)))

# 석식계 학습
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=.2, random_state=42)
model2 = LGBMRegressor(random_state=42)
model2.fit(X_train, y_train)
print("Model2 Score: ",model2.score(X_test, y_test))
print("MAE:",mean_absolute_error(y_test, model2.predict(X_test)))

# 예측
lunch_pred = model1.predict(n_test)
dinner_pred = model2.predict(n_test)

# 제출 문서
submit['중식계'] = lunch_pred
submit['석식계'] = dinner_pred

submit.to_csv('submit.csv', index=False)
pd.read_csv('submit.csv', index_col=False)

100%|██████████| 1255/1255 [00:00<00:00, 199834.92it/s]


Model1 Score:  0.7429368823913827
MAE: 84.55437715555816
Model2 Score:  0.665092001544457
MAE: 46.37569555340799


,일자,중식계,석식계
0,2021-01-27,847.290032,494.881071
1,2021-01-28,958.884907,534.499831
2,2021-01-29,794.935572,534.180665
3,2021-02-01,1353.521916,622.145358
4,2021-02-02,1001.811419,599.336406
5,2021-02-03,690.649417,357.723585
6,2021-02-04,977.828137,541.131623
7,2021-02-05,630.398124,450.437768
8,2021-02-08,1301.736333,650.310418
9,2021-02-09,1082.582458,611.650813


In [33]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5, 7],
    'num_leaves': [31, 61, 91],
}

# 중식계 모델 그리드 서치
model1 = LGBMRegressor(random_state=42)
grid_search1 = GridSearchCV(model1, param_grid, cv=5, n_jobs=-1)
grid_search1.fit(X, y1)
print("Best Parameters: ", grid_search1.best_params_)
best_model1 = grid_search1.best_estimator_

# 석식계 모델 그리드 서치
model2 = LGBMRegressor(random_state=42)
grid_search2 = GridSearchCV(model2, param_grid, cv=5, n_jobs=-1)
grid_search2.fit(X, y2)
print("Best Parameters: ", grid_search2.best_params_)
best_model2 = grid_search2.best_estimator_

# 제출 데이터
pred1 = best_model1.predict(n_test)
pred2 = best_model2.predict(n_test)

submit['중식계'] = pred1
submit['석식계'] = pred2

submit.to_csv('submit.csv', index=False)
pd.read_csv('submit.csv', index_col=False)

Best Parameters:  {'learning_rate': 0.05, 'max_depth': 3, 'num_leaves': 31}
Best Parameters:  {'learning_rate': 0.1, 'max_depth': 3, 'num_leaves': 31}


,일자,중식계,석식계
0,2021-01-27,907.836957,505.387826
1,2021-01-28,925.230132,545.001902
2,2021-01-29,745.716187,517.923881
3,2021-02-01,1253.181244,645.689776
4,2021-02-02,986.129707,605.390861
5,2021-02-03,755.938856,407.235421
6,2021-02-04,940.892255,581.488123
7,2021-02-05,684.667955,437.586506
8,2021-02-08,1231.062061,694.396371
9,2021-02-09,1017.892395,630.313757
